In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import numpy as np
import os

from keras.callbacks import ModelCheckpoint, Callback
from keras.optimizers import Adam
from keras.utils import to_categorical

from networks.unet import Unet

from utils.batch_generator import BatchGenerator

In [3]:
BATCH_SIZE = 1
VAL_BATCH = 10
IMG_ROWS, IMG_COLS = 256, 256
NB_EPOCHS = 10

In [17]:
def mask_to_categorical(masks, batch_size=1):
    masks = to_categorical(masks, 2)
    masks = np.reshape(
        masks,
        (
            batch_size, IMG_COLS * IMG_ROWS, 2
        )
    )
    return masks.astype(np.float32)

In [ ]:
class LossValidateCallback(Callback):

    def __init__(self, batch_generator, results_file):
        self.batch_generator = batch_generator

        basedir = os.path.dirname(results_file)
        if not os.path.exists(basedir):
            os.makedirs(basedir)
        self.results_file = results_file

    @staticmethod
    def IOU_loss(y_true, y_false):
        def IOU_calc(y_true, y_false, smooth=1.):
            y_true_f = y_true.flatten()
            y_false_f = y_false.flatten()
            intersection = np.sum(y_true_f * y_false_f)
            union = np.sum(y_true_f) + np.sum(y_false_f) - intersection
            return (intersection + smooth) / (union + smooth)

        return 1 - IOU_calc(y_true, y_false)

    def on_epoch_end(self, epoch, logs=None):
        train_batch, val_batch = self.batch_generator(VAL_BATCH)
        train_imgs, train_masks = train_batch
        val_imgs, val_masks = val_batch
        train_results = self.model.predict(train_imgs)
        val_results = self.model.predict(val_imgs)

        # change categorical
        train_losses = [
            LossValidateCallback.IOU_loss(
                mask_to_categorical(pair[0]), pair[1]
            )
            for pair in zip(train_masks, train_results)
        ]
        average_train_loss = np.average(train_losses)
        std_train_loss = np.std(train_losses)

        # change categorical
        val_losses = [
            LossValidateCallback.IOU_loss(
                mask_to_categorical(pair[0]), pair[1]
            )
            for pair in zip(val_masks, val_results)
        ]
        average_val_loss = np.average(val_losses)
        std_val_loss = np.std(val_losses)

        # change categorical
        train_size = len(train_masks)
        batch_train_loss = LossValidateCallback.IOU_loss(
            mask_to_categorical(train_masks, train_size), train_results
        )

        # change categorical
        val_size = len(val_masks)
        batch_val_loss = LossValidateCallback.IOU_loss(
            mask_to_categorical(val_masks, val_size), val_results
        )

        eval_train_loss, _ = self.model.evaluate(
            train_imgs, mask_to_categorical(train_masks, train_size)
        )
        eval_val_loss, _ = self.model.evaluate(
            val_imgs, mask_to_categorical(val_masks, val_size)
        )

        text = '{0}, {1}, {2}, '.format(epoch, eval_train_loss, eval_val_loss)
        text += '{0}, {1}, '.format(batch_train_loss, batch_val_loss)
        text += '{0}, {1}, {2}, {3}\n'.format(
            average_train_loss, std_train_loss,
            average_val_loss, std_val_loss
        )
        if not os.path.exists(self.results_file):
            with open(self.results_file, 'w') as file:
                columns = 'epoch, eval_train_loss, eval_validation_loss, '
                columns += 'batch_train_loss, batch_val_loss, '
                columns += 'batch_stats_train_avg_loss, batch_stats_train_std_loss, '
                columns += 'batch_stats_val_avg_loss, batch_stats_val_std_loss\n'
                file.writelines(columns)
            
        with open(self.results_file, 'a') as file:
            file.writelines(text)

In [11]:
def train(data_dir, val_data_dir, results_file):
    batch_gen = BatchGenerator(
        data_dir=data_dir, val_data_dir=val_data_dir, batch_size=BATCH_SIZE
    )
    batch_gen.load_data()
    model = Unet.model(IMG_ROWS, IMG_COLS)
    model.compile(
        optimizer=Adam(lr=1e-4),
        loss='categorical_crossentropy', #Unet.loss
        metrics=[Unet.metric]
    )
    checkpoint = ModelCheckpoint(
        filepath='deep_unet_batch_1_epoch_{epoch:02d}.hdf5',
        mode='auto',
        period=100
    )
    
    def get_batch():
        while True:
            x, mask = next(batch_gen.train_batches)
            mask = to_categorical(mask, 2)
            mask = np.reshape(
                mask,
                (
                    batch_gen.batch_size, IMG_COLS*IMG_ROWS, 2
                )
            )
            yield x, mask
    
    model.fit_generator(
        get_batch(),#batch_gen.train_batches,
        steps_per_epoch=10,
        epochs=NB_EPOCHS,
        callbacks=[
            checkpoint,
            LossValidateCallback(
                batch_gen.generate_test_batch,
                results_file
            )
        ],
    )

In [12]:
data_dir = 'dataset/dataset_256/'
#data_overfit = 'dataset/data_val/'
val_data_dir = 'dataset/val_dataset_256/'
results_file = './results_data.txt'

In [22]:
train(data_dir, val_data_dir, results_file)

Epoch 1/10
10/10 [==============================] - 31s - loss: 5.5188 - metric: -0.6816    
Epoch 2/10
10/10 [==============================] - 19s - loss: 6.3817 - metric: -0.9384    
Epoch 3/10
10/10 [==============================] - 18s - loss: 5.3939 - metric: -0.7783    
Epoch 4/10
10/10 [==============================] - 19s - loss: 1.2730 - metric: 0.2362    
Epoch 5/10
10/10 [==============================] - 19s - loss: 0.7374 - metric: 1.1602    
Epoch 6/10
10/10 [==============================] - 21s - loss: 0.7110 - metric: 1.0929    
Epoch 7/10
10/10 [==============================] - 20s - loss: 0.7013 - metric: 1.2354    
Epoch 8/10
10/10 [==============================] - 18s - loss: 0.7123 - metric: 1.2107    
Epoch 9/10
10/10 [==============================] - 19s - loss: 0.6902 - metric: 1.1903    
Epoch 10/10
10/10 [==============================] - 19s - loss: 0.7038 - metric: 1.1857    


In [43]:
batch_gen = BatchGenerator(
    data_dir=data_dir, val_data_dir=val_data_dir, batch_size=BATCH_SIZE
)
batch_gen.load_data()
model = Unet.model(IMG_ROWS, IMG_COLS)
model.compile(
    optimizer=Adam(lr=1e-4),
    loss=Unet.loss,
    metrics=[Unet.metric]
)

In [44]:
train_batch, val_batch = batch_gen.generate_test_batch(VAL_BATCH)

In [7]:
def IOU_loss(y_true, y_false):
    def IOU_calc(y_true, y_false, smooth=1.):
        y_true_f = y_true.flatten()
        y_false_f = y_false.flatten()
        intersection = np.sum(y_true_f * y_false_f)
        return 2 * (intersection + smooth) / (np.sum(y_true_f) + np.sum(y_false_f) + smooth)

    return 1 - IOU_calc(y_true, y_false)

In [46]:
train_imgs, train_masks = train_batch
val_imgs, val_masks = val_batch

In [55]:
batch_gen = BatchGenerator(
    data_dir=data_dir, val_data_dir=val_data_dir, batch_size=BATCH_SIZE
)
batch_gen.load_data()
model = Unet.model(IMG_ROWS, IMG_COLS)
model.compile(
    optimizer=Adam(lr=1e-4),
    loss=Unet.loss,
    metrics=[Unet.metric]
)
checkpoint = ModelCheckpoint(
    filepath='deep_unet_batch_1_epoch_{epoch:02d}.hdf5',
    mode='auto',
    period=100
)

model.fit_generator(
    batch_gen.train_batches,
    steps_per_epoch=10,
    epochs=10,
    callbacks=[
        checkpoint#,
        #LossValidateCallback(*batch_gen.generate_test_batch(VAL_BATCH), results_file)
    ],
)

Epoch 1/10


ValueError: Cannot feed value of shape (1, 256, 256, 1) for Tensor 'reshape_15_target:0', which has shape '(?, ?, ?)'

In [47]:
train_results = model.predict(train_imgs)
val_results = model.predict(val_imgs)

In [49]:
train_results.shape
val_results.shape

(10, 65536, 2)

In [13]:
IOU_loss(train_masks, train_results)

0.33173472978163765

In [14]:
losses = [IOU_loss(*pair) for pair in zip(train_masks, train_results)]

In [16]:
sum(losses)/len(losses)

0.34012692974544206

In [17]:
losses

[0.3292281621428278,
 0.45997652520413312,
 0.40848355920392054,
 0.21782842908632505,
 0.34723014538066066,
 0.13760194308455564,
 0.36435066638504277,
 0.44095448621814604,
 0.24757870727572417,
 0.44803667347308485]

In [18]:
np.average(losses)

0.34012692974544206

In [19]:
np.std(losses)

0.10302512239756043

In [6]:
batch_gen = BatchGenerator(
    data_dir=data_dir, val_data_dir=val_data_dir, batch_size=BATCH_SIZE
)
batch_gen.load_data()

In [7]:
val = batch_gen.generate_test_batch(VAL_BATCH)

In [16]:
np.unique(next(batch_gen.train_batches)[0])

array([   0.,    1.,    2.,    3.,    4.,    5.,    6.,    7.,    8.,
          9.,   10.,   11.,   12.,   13.,   14.,   15.,   16.,   17.,
         18.,   19.,   20.,   21.,   22.,   23.,   24.,   25.,   26.,
         27.,   28.,   29.,   30.,   31.,   32.,   33.,   34.,   35.,
         36.,   37.,   38.,   39.,   40.,   41.,   42.,   43.,   44.,
         45.,   46.,   47.,   48.,   49.,   50.,   51.,   52.,   53.,
         54.,   55.,   56.,   57.,   58.,   59.,   60.,   61.,   62.,
         63.,   64.,   65.,   66.,   67.,   68.,   69.,   70.,   71.,
         72.,   73.,   74.,   75.,   76.,   77.,   78.,   79.,   80.,
         81.,   82.,   83.,   84.,   85.,   86.,   87.,   88.,   89.,
         90.,   91.,   92.,   93.,   94.,   95.,   96.,   97.,   98.,
         99.,  100.,  101.,  102.,  103.,  104.,  105.,  106.,  107.,
        108.,  109.,  110.,  111.,  112.,  113.,  114.,  115.,  116.,
        117.,  118.,  119.,  120.,  121.,  122.,  123.,  124.,  125.,
        126.,  127.,

In [4]:
from utils.batch_generator import BatchGenerator
BATCH_SIZE = 1
VAL_BATCH = 10
IMG_ROWS, IMG_COLS = 256, 256
NB_EPOCHS = 1001
data_dir = 'dataset/dataset_256/'
data_overfit = 'dataset/data_val/'
val_data_dir = 'dataset/val_dataset_256/'
results_file = './results_data.txt'
batch_gen = BatchGenerator(
    data_dir=data_overfit, val_data_dir=val_data_dir, batch_size=BATCH_SIZE
)

In [5]:
batch_gen.load_data()

In [6]:
next(batch_gen.train_batches)

1


(array([[[[ 244.,  245.,  240.],
          [ 244.,  245.,  240.],
          [ 245.,  246.,  241.],
          ..., 
          [ 224.,  203.,  160.],
          [ 240.,  229.,  209.],
          [ 241.,  241.,  239.]],
 
         [[ 244.,  245.,  240.],
          [ 244.,  245.,  240.],
          [ 245.,  246.,  241.],
          ..., 
          [ 224.,  206.,  160.],
          [ 226.,  218.,  195.],
          [ 242.,  244.,  239.]],
 
         [[ 244.,  245.,  240.],
          [ 244.,  245.,  240.],
          [ 245.,  246.,  241.],
          ..., 
          [ 212.,  196.,  147.],
          [ 215.,  209.,  183.],
          [ 241.,  243.,  238.]],
 
         ..., 
         [[ 246.,  241.,  199.],
          [ 255.,  253.,  208.],
          [ 249.,  245.,  200.],
          ..., 
          [ 240.,  241.,  235.],
          [ 246.,  245.,  243.],
          [ 246.,  242.,  243.]],
 
         [[ 254.,  249.,  207.],
          [ 252.,  248.,  203.],
          [ 246.,  242.,  197.],
          ..., 
  

In [34]:
len(batch_gen._images_pairs)

1